In [ ]:
from keras.applications.vgg19 import VGG19
from keras import layers, models
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import adam_v2
import os
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4,5"

def eachFile(filepath):
	pathDir = os.listdir(filepath)
	out = []
	for allDir in pathDir:
		child = allDir
		out.append(child)
	return out


NUM_CLASSES = 11
TRAIN_PATH = '/Volumes/T7/CCTV_data/Training'
#TRAIN_PATH = '/Volumes/T7/validation/Validation'
VALID_PATH = 'dataset/train'
FC_NUMS = 4096

FREEZE_LAYERS = 17

IMAGE_SIZE = 224

base_model = VGG19(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False
additional_model = models.Sequential()
additional_model.add(base_model)
additional_model.add(layers.Flatten())
additional_model.add(layers.Dense(4096, activation='relu'))
additional_model.add(layers.Dense(2048, activation='relu'))
additional_model.add(layers.Dense(1024, activation='relu'))
additional_model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

additional_model.summary()

additional_model.compile(optimizer=adam_v2.Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
train_datagen = ImageDataGenerator(    
    rescale = 1. /255.,
    validation_split=0.2,
)
train_generator = train_datagen.flow_from_directory(directory=TRAIN_PATH,
                                                    target_size=(IMAGE_SIZE, IMAGE_SIZE), classes=eachFile(TRAIN_PATH), subset='training')
test_generator = train_datagen.flow_from_directory(directory=TRAIN_PATH,
                                                    target_size=(IMAGE_SIZE, IMAGE_SIZE), classes=eachFile(TRAIN_PATH), subset='validation')
print(test_generator.class_indices)
filepath = 'model/sewer_weight.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

history_ft = additional_model.fit_generator(train_generator, epochs=20, callbacks=[checkpoint], 
                                 validation_data=test_generator, validation_steps=5)
additional_model.save('model/sewer_weight_final.h5')


In [ ]:

from keras.utils.generic_utils import CustomObjectScope
from tensorflow.python.keras.initializers import glorot_uniform
from keras.layers import GlobalAveragePooling2D, Dense 
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

print("-- Evaluate --")
#valid_dir = '/Volumes/T7/validation/Validation'
#valid_dir = 'dataset/Testing'
#valid_dir = '/Volumes/T7/CCTV_data/Testing'
valid_dir = 'dataset/sd'
model_path = 'model_test/sewer_weight.h5'
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model(model_path)

IMAGE_SIZE = 224

def eachFile(filepath):
     pathDir = os.listdir(filepath)
     out = []
     for allDir in pathDir:
             child = allDir
             out.append(child)
     return out

valid_datagen = ImageDataGenerator(rescale = 1./255)


# 'valid_dir' 디렉토리에 저장된 이미지 전처리
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), class_mode='categorical',shuffle=False)
print(valid_generator.class_indices)
scores = model.evaluate_generator(valid_generator)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
import numpy as np

test_dir = 'dataset/sd'
test_datagen = ImageDataGenerator(rescale = 1./255)
model_path = 'model_test/sewer_weight.h5'
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model(model_path)

# 'valid_dir' 디렉토리에 저장된 이미지 전처리
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=False)
test_generator.reset()
output = model.predict(test_generator)

test_file_names=test_generator.filenames  
test_labels=test_generator.labels 
class_dict= test_generator.class_indices
print (class_dict) 
new_dict={} 
for key in class_dict: 
    value=class_dict[key]
    new_dict[value]=key
print('PREDICTED CLASS      TRUE CLASS          FILENAME ' )
for i, p in enumerate(output):
    pred_index=np.argmax(p)
    pred_class=new_dict[pred_index] 
    true_class=new_dict[test_labels[i]]
    file=test_file_names[i]
    print(f'       {pred_class}           {true_class}            {file}')

In [19]:
import numpy as np
from keras.utils.generic_utils import CustomObjectScope
from tensorflow.python.keras.initializers import glorot_uniform
from keras.layers import GlobalAveragePooling2D, Dense 
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd

IMAGE_SIZE = 224

test_dir = 'dataset/sd_new'
test_datagen = ImageDataGenerator(rescale = 1./255)
model_path = 'model_test_new/sewer_weight.h5'
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model(model_path)

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), shuffle=False, class_mode = 'categorical')
test_generator.reset()
scores=model.predict_generator(test_generator, len(test_generator))

predicted_class_indices=np.argmax(scores,axis=1)
print(predicted_class_indices)

labels = (test_generator.class_indices)
labelss = dict((v,k) for k,v in labels.items())
predictions = [labelss[k] for k in predicted_class_indices]

filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

print(results)

Found 75 images belonging to 1 classes.


2022-05-27 20:53:05.839937: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


[ 8  8  5  5  8  3  3  8  5  8  8  8  8  8  1  8  7  7  2  7  8  0  0  8
  1  8  8  8  7  8  8  8  8  7  8  9  9  9  9  9  9 10 10  8  5  5  5  1
  8  1  0  5  5  5  5  5  5  8  4  4  4  4  4  9  9  9  9  9  9  2  8  8
  8  7  5]


KeyError: 8

In [ ]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError
from PIL import Image
import os

#folder_path = '/Volumes/T7/CCTV_data/Training/DS'
#folder_path = 'dataset/train/'

folder_path = '/Volumes/T7/validation/Validation/UD_PJ'
print(len(os.listdir(folder_path)))
path = Path(folder_path).rglob("*.png")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)